In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

# Load Data

In [84]:
with open('../data/tangshi_wuyan.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

In [85]:
data=''.join(data)

In [86]:
data[:100]

'唐代五言绝句\n\n王维\n鹿柴\n空山不见人， 但闻人语响。 \n返景入深林， 复照青苔上。 \n\n竹里馆\n\n独坐幽篁里， 弹琴复长啸。 \n深林人不知， 明月来相照。 \n\n送别\n山中相送罢， 日暮掩柴扉。 \n春'

In [87]:
# data I/O
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print(f'data has {data_size} characters, {vocab_size} unique.')
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 8490 characters, 1525 unique.


## Encode text data using one-hot encoding

In [88]:
X_train = np.zeros((len(data), len(chars)))
char_id = np.array([chars.index(c) for c in data])
X_train[np.arange(len(X_train)), char_id] = 1
y_train = np.roll(char_id,-1)

In [89]:
X_train.shape

(8490, 1525)

In [90]:
y_train.shape

(8490,)

# Define some help functions

In [91]:
def get_batch(X_train, y_train, seq_length):
    X = torch.from_numpy(X_train).float()
    y = torch.from_numpy(y_train).long()
    for i in range(0, len(X), seq_length):   
        id_stop = i+seq_length if i+seq_length < len(X) else len(X)
        yield([X[i:id_stop], y[i:id_stop]])

In [92]:
def sample_chars(X_seed, h_prev, length=20):
    X_next = X_seed
    results = []
    with torch.no_grad():
        for i in range(length):        
            y_score, h_prev = rnn(X_next.view(1,1,-1), h_prev)
            y_prob = nn.Softmax(0)(y_score.view(-1)).detach().numpy()
            y_pred = np.random.choice(chars,1, p=y_prob).item()
            results.append(y_pred)
            X_next = torch.zeros_like(X_seed)
            X_next[chars.index(y_pred)] = 1
    return ''.join(results)

# Create a LSTM model

## Define model class

In [93]:
class nn_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
    def forward(self, X, hidden):
        _, hidden = self.lstm(X, hidden)
        output = self.out(hidden[0])
        return output, hidden
    
    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size),
                torch.zeros(1, 1, self.hidden_size)
               )

## Create an instance of the LSTM model

In [94]:
hidden_size = 128
seq_length = 25

In [96]:
rnn = nn_LSTM(vocab_size, hidden_size, vocab_size)

## Define Loss Function

In [97]:
loss_fn = nn.CrossEntropyLoss()

## Define an optimizer

In [98]:
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.005)

## Define a helper training function

In [99]:
def train(X_batch, y_batch):
    h_prev = rnn.initHidden()
    optimizer.zero_grad()
    batch_loss = torch.tensor(0, dtype=torch.float)
    
    for i in range(len(X_batch)):
        y_score, h_prev = rnn(X_batch[i].view(1,1,-1), h_prev)
        loss = loss_fn(y_score.view(1,-1), y_batch[i].view(1))
        batch_loss += loss
    batch_loss.backward()
    # Add parameters' gradients to their values, multiplied by learning rate    
    optimizer.step()

    return y_score, batch_loss/len(X_batch)

In [105]:
# If TensorboardX is installed, we can log and visualize loss in Tensorboard
writer = SummaryWriter(f'logs/lstm1_{time.strftime("%Y%m%d-%H%M%S")}')

In [106]:
all_losses = []
print_every = 100
for epoch in range(20):    
    for batch in get_batch(X_train, y_train, seq_length):
        X_batch, y_batch = batch
        _, batch_loss = train(X_batch, y_batch)
        all_losses.append(batch_loss.item())
        if len(all_losses)%print_every==1:
            print(f'----\nRunning Avg Loss:{np.mean(all_losses[-print_every:])} at iter: {len(all_losses)}\n----')
            writer.add_scalar('loss', np.mean(all_losses[-100:]), len(all_losses))
            print(sample_chars(X_batch[0], rnn.initHidden(), 200))

----
Running Avg Loss:0.7109438180923462 at iter: 1
----
代五言绝句

蜗牛
腥涎不满壳，聊闻芦村朝，秋风多钱绝。

相忆南所思，庭尽手头一。
但见泪痕湿，山南第一花。

送别夜上定，昨夜东江雨。
长料鼠与已，坐待碧波中。

杨雯
北客
清旷天所依，曾向雨中忘。
灞桥风雨夜，年光观东东。

柯芝
横江
横江一片碧，携将万黄原。
谁谓物无名，云檐复濯思。
悠然钓乡畔，邻客坐落冷。

马知节
枯梅
床前溪水路，阴黄生水黛。
拙于用显晦，踪便照酴蘼。

旧过

----
Running Avg Loss:0.6730500864982605 at iter: 101
----
仲令
绝句
吴李叔
井游空山不剪时红清速，恋恋忽复稀。
烂日到扬发，君江清子数。

司马光
野江
聊行弄芳草，今不看嫌心。

岐山宦阁雨，欲原双泪横。
粉身能报国，借与但山海。

赵师侠
李雁
无台植
野元
寥卧九相照，东日暮复床。
仙女瑟瑟衣，寒尽不知年。

况周遇风
云业转林色，未待月黄昏。

送林行
刘子元
苍阳不禁冷，破褐出残絮。
野性岂欲下，只是乱归来。

昭君 明·杨灵，秋江裙带漪。 
----
Running Avg Loss:0.618951216340065 at iter: 201
----
听。

周孚
公佐索诗
堆浸石根冷，风高处数飞。
欲怜湖上山，鸥鹭亦迷群。

贺铸
泠沼
明月照齐州，花光被动曲。
不见棹回人，明月驹隙过。

蚕
斫水水可赵，续弦弦可完。

谓物老江水色，照深应断花。

左纬
许少弄德高，曾秋去年雪。

短棹风生，何许小残曲。
俗人诗飞里，残余频泪滴。
松草多稚旧，马就平沙涉。
不在。飞事，留取隔帘看。

范成大
双燕
底处双飞鸳，江雪见南去。

周阿崇
题绿波
----
Running Avg Loss:0.62886733725667 at iter: 301
----

秋载干出矣，上藏亦英华。
翻许傍去不，山云不劝归。

采君
北夜几烟恨，千叶乾红寒。

收茵
自题失山鹤， 静闻小语红。

昭君石足色双，花开一春风。
如何临欲别，只待妾衣裳。

杨杰
题余杭步伍亭壁
欲外秋已晚， 遗钓归山雪。 

高风吹古乐，宛如发为人。
只在眠如发，邻过杏花来。

黄金无足色，山泪如执中。

龙栖欲寻住，西

In [107]:
print(sample_chars(X_batch[0], rnn.initHidden(), 2000))

燕子知时节，还从旧宇归。
新人方按曲，不许傍檐飞。

刘韬玉
至角黄之出，韬藏亦英华。
余香被草木，秀擢蜀长长。

丽泽
长哦伐木篇，蜂忙供蜜工。
役役为人劳，衣食还自空。

刘克庄
宿春
西窗雨气浓，秋红蜻偷碧。
妾当年少梦，清夜音南飞。

段明
梅花
微汝本月程，不买千三开。
田家临水机，不便照酴居。

离?州作
相看不忍发，惨淡弦为去。

李诚之
葵日鸟飞绝，银钩楷法精。
啼应秋风下，一意已如许。
云影寒宿一，时有还自去。
寸心即知已，不用世人知。

刘文晦
种梅
佳人天一方，岁暮音书绝。
当处芳草多，相呼向深坞。
竹外立寒枝，山紫又复空。

裳扇
临水
门天虽自献，破褐出残絮。
童儿不耐烦，微裘拥绣鞍。

酬昭君怨 元·赵介
一身归朔漠，数代去不归。

朱子
莲沼
亭亭玉芙蓉，迥立映澄碧。

钓台
穷鳞无处伤，过尽上头眠。

张至龙
演雅溪
和刘卿湖
屏生不县希，斜林未是愁。
过尽前头望，又得夜来雨。

朝食山上
恨后湖随形鲤
日暮飞鸟归，门前长春水。

濯清亭
庭衣回
闻生各城晨，门前长春水。

濯清亭
春风记维舟，春子破长身。
田儿愁山中，时时倚藜杖。

斯植
古乐府
片片落花飞，随风过南浦。

刘克庄
宿建德江
岭外音书绝， 经冬逢愁。
移争小林头， 明月来生少。 

韦应物
秋生，高斋卧看山。

黄金无足色，山上人知动。

薛峪
蓑衣步
水浸石根冷，风吹藤叶飞。

严肃
落花
片片落花飞，随香被草死。
想是一山下，摇思无定林。

刘克庄
宿建德江
移舟泊烟渚，秋江入燕飞。
别来看成下，不忧世空林。

高翥
题雷公井
霭音书于俗，千里无雪古。
不道山中冷，翻忧世上人。

丽泽
长哦伐木篇，园林未是归。
何郑公客梦，到蜀到钱唐。

牟岘
鹿柴

云议隐厨主，高枕石壁。
但鸣随船轻，不知今古春。
不忧雷雨作，头角本非真。

大慈野花望，山清望余知。
如何临欲别，不得傍君衣。

王曼之
西窗
西窗枕寒池，池边老松树。
渴猿不知年，不在古书中。

杨杰
何满子
和浦望九韵
春风老飞灵，落叶两叶坠。

杨杰
万里色苍然，寒林夕照边。
旧声何满子，清林开数茎。

赵师侠
梅花
南国觅佳人，风回亦逢春。

苏辙
璎国罗亭
山云泽气浓，曾将热长春。

陈允平
山中吟
东望泰岳松， 不敢过临洮。 

唐庚
栖禅暮归书书鱼
黄卧提幅树，觉林不可完。

追和亡景遇
题卧道江